<a href="https://colab.research.google.com/github/liannepob/finalproject/blob/main/Poblador_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import random

# Base Character class
class Character:
    def __init__(self, name, health):
        self.name = name
        self.health = health

# Enemy class that inherits from Character
class Enemy(Character):
    def __init__(self, name, health, damage):
        super().__init__(name, health)
        self.damage = damage

    def attack(self):
        return random.randint(1, self.damage)

# NPC (Non-Playable Character) class that inherits from Character
class NPC(Character):
    def __init__(self, name, health, message):
        super().__init__(name, health)
        self.message = message

    def talk(self):
        return self.message

# Player class to manage player-specific information
class Player(Character):
    def __init__(self, name, health=100, location="Welcome_Room"):
        super().__init__(name, health)
        self.inventory = []
        self.location = location

    def move(self, direction):
        exits = rooms[self.location]["exits"]
        for key in exits:
            if key.lower() == direction.lower():
                self.location = exits[key]
                print(f"You move {direction} to {self.location}.")
                return
        print("You can't go that way.")

    def pick_up(self, item):
        self.inventory.append(item)
        print(f"\nYou picked up {item}.")
        if item in rooms[self.location]["items"]:
            rooms[self.location]["items"].remove(item)

    def show_status(self):
        print(f"\n=== STATUS ===")
        print(f"Name: {self.name}")
        print(f"Health: {self.health}")
        print(f"Location: {self.location}")
        print(f"Inventory: {self.inventory}")
        print("================")

# Game world setup (a simple room-based map)
rooms = {
    "Welcome_Room": {
        "description": "A bright white room with a glowing welcome sign.",
        "items": ["key"],
        "exits": {"South": "Main_Hallway"},
        "enemy": None,
        "npc": NPC("Dog", 10, "Good luck ahead! Woof!")
    },
    "Main_Hallway": {
        "description": "A dark hallway with creaky wood and spiderwebs.",
        "items": [],
        "exits": {
            "North": "Welcome_Room",
            "East": "Treasure_Room",
            "South": "Greenhouse",
            "West": "Potion_Room"
        },
        "enemy": Enemy("Zombie", 30, 5),
        "npc": NPC("Cat", 10, "Beware of what's lurking.")
    },
    "Treasure_Room": {
        "description": "A room filled with glittering gold and jewels.",
        "items": ["gold", "ruby"],
        "exits": {"West": "Main_Hallway"},
        "enemy": None,
        "npc": None
    },
    "Greenhouse": {
        "description": "A peaceful room with flowers and butterflies.",
        "items": ["healing herb"],
        "exits": {"North": "Main_Hallway"},
        "enemy": None,
        "npc": NPC("Gardener", 20, "These herbs can restore health.")
    },
    "Potion_Room": {
        "description": "A spooky room full of potions and glass bottles.",
        "items": ["potion"],
        "exits": {"East": "Main_Hallway"},
        "enemy": Enemy("Witch", 40, 8),
        "npc": NPC("Old Goat", 15, "Don't trust the Witch!")
    }
}

# Saves the current state of the player to a JSON file
def save_game(player):
    data = {
        "name": player.name,
        "health": player.health,
        "inventory": player.inventory,
        "location": player.location
    }
    with open("savegame.json", "w") as f:
        json.dump(data, f, indent=2)
    print("Game saved.")

# Loads the player state from the saved JSON file
def load_game():
    try:
        with open("savegame.json", "r") as f:
            data = json.load(f)
        player = Player(data["name"], data["health"], data["location"])
        player.inventory = data["inventory"]
        print("Game loaded.")
        return player
    except FileNotFoundError:
        print("No save found.")
        return None

# Main game loop
def main():
    print("Welcome to the Adventure Game!")
    name = input("Enter your name: ")
    player = Player(name)

    while True:
        current_room = rooms[player.location]
        print(f"\nYou are in {player.location}: {current_room['description']}")

        if current_room["npc"]:
            print(f"{current_room['npc'].name} says: {current_room['npc'].talk()}")

        for item in list(current_room["items"]):
            if item not in player.inventory:
                pick = input(f"Do you want to pick up {item}? (yes/no) ").strip().lower()
                if pick == "yes":
                    player.pick_up(item)

        if current_room["enemy"]:
            enemy = current_room["enemy"]
            print(f"A {enemy.name} appears! Health: {enemy.health}")
            while enemy.health > 0:
                action = input("Attack or run? ").strip().lower()
                if action == "attack":
                    damage = random.randint(5, 15)
                    enemy.health -= damage
                    print(f"You attack the {enemy.name} for {damage} damage.")
                    if enemy.health <= 0:
                        print(f"You defeated the {enemy.name}!")
                        rooms[player.location]["enemy"] = None  # Enemy removed
                        break
                    else:
                        enemy_damage = enemy.attack()
                        player.health -= enemy_damage
                        print(f"{enemy.name} attacks you for {enemy_damage} damage!")
                        if player.health <= 0:
                            print("You died. Game over.")
                            return
                elif action == "run":
                    print("You escaped safely.")
                    break
                else:
                    print("Invalid action.")

        if player.location == "Greenhouse":
            regen = 5
            player.health += regen
            print(f"You feel refreshed. +{regen} health.")

        # Player input
        choice = input("\nWhat now? (move/save/load/status/quit) ").strip().lower()
        if choice == "move":
            direction = input("Direction (north/south/east/west): ").strip().lower()
            player.move(direction)
        elif choice == "save":
            save_game(player)
        elif choice == "load":
            new_player = load_game()
            if new_player:
                player = new_player
        elif choice == "status":
            player.show_status()
        elif choice == "quit":
            print("Thank you for playing!")
            break
        else:
            print("Unknown command.")

if __name__ == "__main__":
    main()

Welcome to the Adventure Game!
Enter your name: Lianne

You are in Welcome_Room: A bright white room with a glowing welcome sign.
Dog says: Good luck ahead! Woof!
Do you want to pick up key? (yes/no) yes

You picked up key.

What now? (move/save/load/status/quit) move
Direction (north/south/east/west): south
You move south to Main_Hallway.

You are in Main_Hallway: A dark hallway with creaky wood and spiderwebs.
Cat says: Beware of what's lurking.
A Zombie appears! Health: 30
Attack or run? attack
You attack the Zombie for 14 damage.
Zombie attacks you for 2 damage!
Attack or run? attack
You attack the Zombie for 8 damage.
Zombie attacks you for 2 damage!
Attack or run? attack
You attack the Zombie for 15 damage.
You defeated the Zombie!

What now? (move/save/load/status/quit) move
Direction (north/south/east/west): east
You move east to Treasure_Room.

You are in Treasure_Room: A room filled with glittering gold and jewels.
Do you want to pick up gold? (yes/no) yes

You picked up gold